In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
df=pd.read_csv('./parsed data/table.csv')
if 'is_spam' in df:
    y = df['is_spam'].values # get the labels we want
    del df['is_spam'] # get rid of the class label
    X=df.values

sc =StandardScaler()

#normalize data
X=sc.fit_transform(X)
#train-test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Split X_train again to create validation data
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.2)
print(X)

[[ 0.14891747  7.67534088 -0.6070791  ..., -0.07735751 -0.03178106
   0.94336346]
 [ 0.49891483  7.16352571 -0.6070791  ..., -0.06374392  0.42255723
   0.94336346]
 [-0.44170307  7.15748682 -0.6070791  ..., -0.08295187  0.42255723
   0.94336346]
 ..., 
 [ 0.01766846 -1.18371046 -0.6070791  ..., -0.06475587  0.42255723
   0.94336346]
 [ 0.14891747 -1.18238553 -0.6070791  ..., -0.0713049   0.42255723
  -1.06003681]
 [ 0.1707923  -1.18478153 -0.6070791  ..., -0.07527633 -2.64963504
  -1.06003681]]


/Users/arslanmemon/miniconda2/envs/MLEnv/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


# Modeling

## MODEL 1 

(PURPOSE)

(CODE)

(HYPERTUNE PARAMETER) 

## MODEL 2